In [1]:
import pandas as pd
from gensim import corpora, models
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize

In [91]:
train_df = pd.read_csv('sample_train.csv')

In [92]:
train_df['documents'] = train_df['documents'].apply(lambda x: eval(x))

In [93]:
import re

def clean(df: pd.DataFrame) -> pd.DataFrame:
    # convert to lowercase
    df['documents'] = df['documents'].apply(lambda x: [i.lower() for i in x])
    # remove special characters
    df['documents'] = df['documents'].apply(lambda x: [re.sub(r'[^a-zA-Z0-9!.?]', ' ', i) for i in x])
    # remove multiple spaces
    df['documents'] = df['documents'].apply(lambda x: [re.sub(r' +', ' ', i) for i in x])
    # Lemmatize
    lmtzr = WordNetLemmatizer()
    df['documents'] = df['documents'].apply(lambda x: [lmtzr.lemmatize(i) for i in x])
    return df

In [94]:
clean_df = clean(train_df.copy(deep=True))

In [99]:
train_df

,documents,num_documents,summary
0,[A Ware Police patrol car passes the yellow ho...,4,– A defrocked priest at the heart of Boston's ...
1,[Significance \n \n A large body of medical re...,5,– Women suffering from a heart attack seem to ...
2,[A Syrian warplane has destroyed a petrol stat...,4,"– A new UN analysis finds that at least 60,000..."
3,"[A political cartoon, published in a newspaper...",7,– The first government shutdown since the Clin...
4,[Megyn Kelly takes issue with 'NYT' book revie...,4,"– Megyn Kelly's memoir is out next week, and s..."
...,...,...,...
495,[Noah Walker has been taken into custody. He w...,6,– One of the 170 bikers arrested after Sunday'...
496,[Rice: White House Summit To Show Africa's Pot...,6,"– This week, Washington is hosting most of Afr..."
497,[Story highlights Images appear to show new ai...,4,– China is engaged in something more than mere...
498,[Media reports abound that US President Donald...,6,– US relations with China took a potentially n...


In [95]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()

documents = set()

for one_topic in clean_df['documents']:
    documents.update(one_topic)

tfidf_matrix = tfidf.fit_transform(documents)

In [96]:
from sklearn.cluster import KMeans

num_clusters = 3

kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(tfidf_matrix)

clean_df['topic'] = clean_df['documents'].apply(lambda x: kmeans.predict(tfidf.transform(x)))

/home/shu7bh/.conda/envs/main/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [142]:
groups = []
similar_df = train_df.copy(deep=True)
for i in range(clean_df.shape[0]):
    grouped_doc.clear()
    grouped_doc = {i: [] for i in range(num_clusters)}
    for idx, j in enumerate(clean_df['topic'][i]):
        grouped_doc[j].append(train_df['documents'][i][idx])
    max_topic = max(grouped_doc, key=lambda x: len(grouped_doc[x]))
    groups.append(grouped_doc[max_topic])

In [143]:
similar_df['documents'] = groups
similar_df['num_documents'] = similar_df['documents'].apply(lambda x: len(x))

In [144]:
similar_df

,documents,num_documents,summary
0,[A Ware Police patrol car passes the yellow ho...,3,– A defrocked priest at the heart of Boston's ...
1,[Female heart attack patients treated by male ...,4,– Women suffering from a heart attack seem to ...
2,[A Syrian warplane has destroyed a petrol stat...,4,"– A new UN analysis finds that at least 60,000..."
3,"[A political cartoon, published in a newspaper...",6,– The first government shutdown since the Clin...
4,[We've detected that JavaScript is disabled in...,2,"– Megyn Kelly's memoir is out next week, and s..."
...,...,...,...
495,[Noah Walker has been taken into custody. He w...,3,– One of the 170 bikers arrested after Sunday'...
496,[Rice: White House Summit To Show Africa's Pot...,5,"– This week, Washington is hosting most of Afr..."
497,[Story highlights Images appear to show new ai...,2,– China is engaged in something more than mere...
498,[Media reports abound that US President Donald...,5,– US relations with China took a potentially n...


In [146]:
similar_df.to_csv('tf_idf_train.csv')